In [ ]:
"""
An example of using the stack() and unstack() methods to take a tabluar data strucure and "flip" it into an EAV structure.
It does this my maninpulating the structure of data, not the data itself. Hierarchical indexes are created and manipulated.

Think of "stacking" as creating longer thinner tables, and "unstacking" as creating shorter fatter tables.

A copy of patients_1.xlxs is in the same directory for demo purposes. However, depending your setup you may have to set your
python environment to the appropriate directory. For example:

import os
os.getcwd() ## check current directory
os.chdir("src/test_data") # navigate relative to your directory structure

Web pages for more information about stack()/unstack() and hierarchical indexing:

http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/
https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77
https://pandas.pydata.org/pandas-docs/stable/advanced.html
http://www.datasciencemadesimple.com/reshape-using-stack-unstack-function-pandas-python/
https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html

"""

In [2]:
import pandas as pds

# load Excel file into dataframe
patients = pds.ExcelFile('patients_1.xlsx').parse()
patients

,patient_id,gender,birth_date
0,10001,M,1950-01-01
1,10002,F,1960-01-02
2,10003,M,1970-01-03
3,10004,F,1980-01-04
4,10005,M,1990-01-05
5,10006,F,1955-01-06
6,10007,M,1965-01-07
7,10008,F,1975-01-08
8,10009,M,1985-01-09
9,10010,F,1995-01-10


In [3]:
# by default the index does not have a name
# so, set index name to 'record'
# note: in the output 'record' is not a column, but an index
# you can see this visually b/c it is "lower" than the column names
patients.index.name = 'record'
patients

,patient_id,gender,birth_date
record,,,
0,10001,M,1950-01-01
1,10002,F,1960-01-02
2,10003,M,1970-01-03
3,10004,F,1980-01-04
4,10005,M,1990-01-05
5,10006,F,1955-01-06
6,10007,M,1965-01-07
7,10008,F,1975-01-08
8,10009,M,1985-01-09


In [4]:
# the patients in the dataset need to be a associated with a practice
# conceptually, this is similar to each record being 
# linked to a project in the REDCap data

# add practice info
patients['practice'] = 1 
patients

,patient_id,gender,birth_date,practice
record,,,,
0,10001,M,1950-01-01,1
1,10002,F,1960-01-02,1
2,10003,M,1970-01-03,1
3,10004,F,1980-01-04,1
4,10005,M,1990-01-05,1
5,10006,F,1955-01-06,1
6,10007,M,1965-01-07,1
7,10008,F,1975-01-08,1
8,10009,M,1985-01-09,1


In [5]:
# before creating EAV structure some renaming and conversion
# operations are needed to on dataframe structure

# turn indexes into columns
patients.reset_index(inplace=True)
patients

,record,patient_id,gender,birth_date,practice
0,0,10001,M,1950-01-01,1
1,1,10002,F,1960-01-02,1
2,2,10003,M,1970-01-03,1
3,3,10004,F,1980-01-04,1
4,4,10005,M,1990-01-05,1
5,5,10006,F,1955-01-06,1
6,6,10007,M,1965-01-07,1
7,7,10008,F,1975-01-08,1
8,8,10009,M,1985-01-09,1
9,9,10010,F,1995-01-10,1


In [6]:
# reorder for convenience 
patients = patients[['practice', 'record', 'patient_id', 'gender', 'birth_date']] 
patients

,practice,record,patient_id,gender,birth_date
0,1,0,10001,M,1950-01-01
1,1,1,10002,F,1960-01-02
2,1,2,10003,M,1970-01-03
3,1,3,10004,F,1980-01-04
4,1,4,10005,M,1990-01-05
5,1,5,10006,F,1955-01-06
6,1,6,10007,M,1965-01-07
7,1,7,10008,F,1975-01-08
8,1,8,10009,M,1985-01-09
9,1,9,10010,F,1995-01-10


In [7]:
# before using stack(), indexes are needed to use stack() and unstack() operations
# see: http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

# set indexes to practice and record; this results in a hierarchical index
# notice that practice and record columns are now indexes (visually "lower" than other column names)
patients.set_index(['practice', 'record'], inplace=True)
patients

patient_id gender  birth_date
practice record                               
1        0            10001      M  1950-01-01
         1            10002      F  1960-01-02
         2            10003      M  1970-01-03
         3            10004      F  1980-01-04
         4            10005      M  1990-01-05
         5            10006      F  1955-01-06
         6            10007      M  1965-01-07
         7            10008      F  1975-01-08
         8            10009      M  1985-01-09
         9            10010      F  1995-01-10

In [8]:
# the stack() operation will take the column names and create indexes from them
# use stack to create EAV dataframe
eav = patients.stack()
eav

practice  record            
1         0       patient_id         10001
                  gender                 M
                  birth_date    1950-01-01
          1       patient_id         10002
                  gender                 F
                  birth_date    1960-01-02
          2       patient_id         10003
                  gender                 M
                  birth_date    1970-01-03
          3       patient_id         10004
                  gender                 F
                  birth_date    1980-01-04
          4       patient_id         10005
                  gender                 M
                  birth_date    1990-01-05
          5       patient_id         10006
                  gender                 F
                  birth_date    1955-01-06
          6       patient_id         10007
                  gender                 M
                  birth_date    1965-01-07
          7       patient_id         10008
                  gender 

In [9]:
# now we need to take the indexes and turn them into columns
# the new indexes do not have any names associated with them
# so the output column names are created algorithmically (e.g., "level_2", "0")

# convert indexes to column names
eav = eav.reset_index()
eav

,practice,record,level_2,0
0,1,0,patient_id,10001
1,1,0,gender,M
2,1,0,birth_date,1950-01-01
3,1,1,patient_id,10002
4,1,1,gender,F
5,1,1,birth_date,1960-01-02
6,1,2,patient_id,10003
7,1,2,gender,M
8,1,2,birth_date,1970-01-03
9,1,3,patient_id,10004


In [10]:
# set meaningful EAV column names
eav.columns = ['practice', 'record', 'field', 'value']
eav

,practice,record,field,value
0,1,0,patient_id,10001
1,1,0,gender,M
2,1,0,birth_date,1950-01-01
3,1,1,patient_id,10002
4,1,1,gender,F
5,1,1,birth_date,1960-01-02
6,1,2,patient_id,10003
7,1,2,gender,M
8,1,2,birth_date,1970-01-03
9,1,3,patient_id,10004


In [ ]:
# save EAV table (skip saving for now)
## eav.to_csv(index=False)

In [13]:
# now unstack the EAV to get the tabular format back

# create indexes onthe practice, record, and field columns
eav_idx = eav.set_index(['practice', 'record', 'field'])
eav_idx

value
practice record field                 
1        0      patient_id       10001
                gender               M
                birth_date  1950-01-01
         1      patient_id       10002
                gender               F
                birth_date  1960-01-02
         2      patient_id       10003
                gender               M
                birth_date  1970-01-03
         3      patient_id       10004
                gender               F
                birth_date  1980-01-04
         4      patient_id       10005
                gender               M
                birth_date  1990-01-05
         5      patient_id       10006
                gender               F
                birth_date  1955-01-06
         6      patient_id       10007
                gender               M
                birth_date  1965-01-07
         7      patient_id       10008
                gender               F
                birth_date  1975-01-08
         8      patient_id       10009
                gender               M
                birth_date  1985-01-09
         9      patient_id       10010
                gender               F
                birth_date  1995-01-10

In [15]:
# perform unstack operation
# this will shift the field index to be column
eav_unstack = eav_idx.unstack()
eav_unstack

value                  
field            birth_date gender patient_id
practice record                              
1        0       1950-01-01      M      10001
         1       1960-01-02      F      10002
         2       1970-01-03      M      10003
         3       1980-01-04      F      10004
         4       1990-01-05      M      10005
         5       1955-01-06      F      10006
         6       1965-01-07      M      10007
         7       1975-01-08      F      10008
         8       1985-01-09      M      10009
         9       1995-01-10      F      10010

In [22]:
# reset the indexes for practice and record to turn them into columns
# IMPORANT: use col_level=1 to place practice and record on the same level as other columns
tabular = eav_unstack.reset_index(['practice','record'], col_level=1)
tabular

value                  
field practice record  birth_date gender patient_id
0            1      0  1950-01-01      M      10001
1            1      1  1960-01-02      F      10002
2            1      2  1970-01-03      M      10003
3            1      3  1980-01-04      F      10004
4            1      4  1990-01-05      M      10005
5            1      5  1955-01-06      F      10006
6            1      6  1965-01-07      M      10007
7            1      7  1975-01-08      F      10008
8            1      8  1985-01-09      M      10009
9            1      9  1995-01-10      F      10010

In [23]:
# change all column headings to show only bottom level
tabular.columns = [col[1] for col in tabular.columns]
tabular

,practice,record,birth_date,gender,patient_id
0,1,0,1950-01-01,M,10001
1,1,1,1960-01-02,F,10002
2,1,2,1970-01-03,M,10003
3,1,3,1980-01-04,F,10004
4,1,4,1990-01-05,M,10005
5,1,5,1955-01-06,F,10006
6,1,6,1965-01-07,M,10007
7,1,7,1975-01-08,F,10008
8,1,8,1985-01-09,M,10009
9,1,9,1995-01-10,F,10010


In [25]:
# reset the indexes so that columns are column names (not indexes)
# drop=True prevents a new index from being created
tabular.reset_index(drop=True, inplace=True)
tabular

,practice,record,birth_date,gender,patient_id
0,1,0,1950-01-01,M,10001
1,1,1,1960-01-02,F,10002
2,1,2,1970-01-03,M,10003
3,1,3,1980-01-04,F,10004
4,1,4,1990-01-05,M,10005
5,1,5,1955-01-06,F,10006
6,1,6,1965-01-07,M,10007
7,1,7,1975-01-08,F,10008
8,1,8,1985-01-09,M,10009
9,1,9,1995-01-10,F,10010


In [ ]:
## Fine